# 2-4 Segmentation

In [ ]:
!pip3 install --upgrade mxnet-cu101 > /dev/null
!pip3 install --upgrade gluoncv > /dev/null

## 테스트이미지
- <img src='https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true' /> 

- <img src='https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true' width=640 /> 

- <img src='https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true' />

## Instance Segmentation
- Mask RCNN

In [ ]:
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils

### Mask RCNN Framework
<img src='https://www.researchgate.net/publication/337746202/figure/fig2/AS:832550465449984@1575506907034/The-Framework-of-Mask-R-CNN-55.png' width=100% />

### model loading

In [ ]:
net = model_zoo.get_model('mask_rcnn_resnet50_v1b_coco', pretrained=True)

### image download

In [ ]:
im_fname = utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true',   path='biking.jpg')
x, orig_img = data.transforms.presets.rcnn.load_test(im_fname)

### inference and display

In [ ]:
ids, scores, bboxes, masks = [xx[0].asnumpy() for xx in net(x)]

# paint segmentation mask on images directly
width, height = orig_img.shape[1], orig_img.shape[0]
masks, _ = utils.viz.expand_mask(masks, bboxes, (width, height), scores)
orig_img = utils.viz.plot_mask(orig_img, masks)

# identical to Faster RCNN object detection
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax = utils.viz.plot_bbox(orig_img, bboxes, scores, ids,
                         class_names=net.classes, ax=ax)
plt.show()

### 처리 함수 정의

In [ ]:
def display_instance_segmentation(model_name, image_filename):
  import time
  from matplotlib import pyplot as plt
  from gluoncv import model_zoo, data, utils

  net = model_zoo.get_model(model_name, pretrained=True)

  tic = time.time()
  x, orig_img = data.transforms.presets.rcnn.load_test(image_filename)

  ids, scores, bboxes, masks = [xx[0].asnumpy() for xx in net(x)]

  # paint segmentation mask on images directly
  width, height = orig_img.shape[1], orig_img.shape[0]
  masks, _ = utils.viz.expand_mask(masks, bboxes, (width, height), scores)
  orig_img = utils.viz.plot_mask(orig_img, masks)

  # identical to Faster RCNN object detection
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax = utils.viz.plot_bbox(orig_img, bboxes, scores, ids,
                          class_names=net.classes, ax=ax)
  plt.show()
  
  print('process time : {}'.format(time.time() - tic))

### image test

In [ ]:
im_fname = utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')
display_instance_segmentation('mask_rcnn_resnet50_v1b_coco', im_fname)

In [ ]:
im_fname = utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_instance_segmentation('mask_rcnn_resnet50_v1b_coco', im_fname)

In [ ]:
im_fname = utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_instance_segmentation('mask_rcnn_resnet50_v1b_coco', im_fname)

### [Mark-RCNN-Shiny](https://github.com/huuuuusy/Mask-RCNN-Shiny)
<img src='https://github.com/huuuuusy/Mask-RCNN-Shiny/blob/master/demo/nikki.gif?raw=true' />

## Semantic Segmentation - FCN
- Fully Convolutional Networks

### FCN Framework
<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fc2rRKu%2FbtqvFhluGR4%2FL0t8Da4wcUNJzkxPmzH6tk%2Fimg.jpg' />

In [ ]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
# using cpu
ctx = mx.cpu(0) 

In [ ]:
model = gluoncv.model_zoo.get_model('fcn_resnet101_voc', pretrained=True)

### FCN Layers

In [ ]:
model.summary

### Image Download

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')

### image pre transforming

In [ ]:
img = image.imread(im_fname)

from gluoncv.data.transforms.presets.segmentation import test_transform
img = test_transform(img, ctx)

In [ ]:
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

In [ ]:
from gluoncv.utils.viz import get_color_pallete
import matplotlib.image as mpimg
mask = get_color_pallete(predict, 'pascal_voc')
mask.save('output.png')

In [ ]:
type(mask)

In [ ]:
mmask = mpimg.imread('output.png')
plt.imshow(mmask)
plt.show()

### 처리 함수 정의

In [ ]:
def display_semantic_segmentation(model_name, image_filename, color_pallete='pascal_voc'):
  import time
  import mxnet as mx
  from mxnet import image
  from mxnet.gluon.data.vision import transforms
  from matplotlib import pyplot as plt
  import gluoncv
  from gluoncv.utils.viz import get_color_pallete
  import matplotlib.image as mpimg

  # using cpu
  ctx = mx.cpu(0) 

  # model download
  model = gluoncv.model_zoo.get_model(model_name, pretrained=True)

  tic = time.time()
  # image loading
  img = image.imread(image_filename)

  # image pre transforming
  from gluoncv.data.transforms.presets.segmentation import test_transform
  img = test_transform(img, ctx)

  # inference
  output = model.predict(img)
  predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

  # display

  mask = get_color_pallete(predict, color_pallete)
  mask.save('output.png')

  mmask = mpimg.imread('output.png')
  plt.imshow(mmask)
  plt.show()
  
  print('process time : {}'.format(time.time() - tic))


### image test

In [ ]:
import gluoncv

In [ ]:
im_fname = utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')
display_semantic_segmentation('fcn_resnet101_voc', im_fname)

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation('fcn_resnet101_voc', im_fname)

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_semantic_segmentation('fcn_resnet101_voc', im_fname)

## Semantic Segmentation -PSPNet
- Pyramid Scene Parsing network

### PSP Network Framework
<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fba73Zm%2Fbtqwl64dcBn%2FV6cr71k8IvxQ71J9MmfrM1%2Fimg.png' width=100% />

In [ ]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
# using cpu
ctx = mx.cpu(0)

In [ ]:
model = gluoncv.model_zoo.get_model('psp_resnet101_ade', pretrained=True)

### psp network layer

In [ ]:
model.summary

### image download

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')

In [ ]:
img = image.imread(im_fname)

### image transform

In [ ]:
from gluoncv.data.transforms.presets.segmentation import test_transform
img = test_transform(img, ctx)

### inference

In [ ]:
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

In [ ]:
from gluoncv.utils.viz import get_color_pallete
import matplotlib.image as mpimg
mask = get_color_pallete(predict, 'ade20k')
mask.save('output.png')

In [ ]:
from matplotlib import pyplot as plt
mmask = mpimg.imread('output.png')
plt.imshow(mmask)
plt.show()

### image test

In [ ]:
im_fname = utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')
display_semantic_segmentation('psp_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation('psp_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_semantic_segmentation('psp_resnet101_ade',im_fname, 'ade20k')

## Semantic Segmentation - DeepLabV3

In [ ]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
from matplotlib import pyplot as plt

# using cpu
ctx = mx.cpu(0)

In [ ]:
model = gluoncv.model_zoo.get_model('deeplab_resnet101_ade', pretrained=True)

### Deeplab framework
<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcw5eBB%2FbtqBMNYTJYZ%2FFhQ4uwqp6TQeotgozZOIlk%2Fimg.png' />

### deeplab layer

In [ ]:
model.summary

### image download

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')

In [ ]:
img = image.imread(im_fname)

### image pre transforming

In [ ]:
from gluoncv.data.transforms.presets.segmentation import test_transform
img = test_transform(img, ctx)

### inference

In [ ]:
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

### display image

In [ ]:
from gluoncv.utils.viz import get_color_pallete
import matplotlib.image as mpimg
mask = get_color_pallete(predict, 'ade20k')
mask.save('output.png')

In [ ]:
mmask = mpimg.imread('output.png')
plt.imshow(mmask)
plt.show()

### image test

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/2516944023_d00345997d_z.jpg?raw=true')
display_semantic_segmentation('deeplab_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation('deeplab_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_semantic_segmentation('deeplab_resnet101_ade',im_fname, 'ade20k')

## 실습
- 아래의 이미지를 각각의 모델로 분석하세요
- instance Segmentation(Mask RCNN)
- Semantic Segmentation(FCN, PSPNet, DeepLabv3)

<img src='https://imgnews.pstatic.net/image/077/2020/12/17/PAP20201217074101055_P2_20201217055630293.jpg' />

## 이미지에 마스크 overlay

In [ ]:
def display_semantic_segmentation_mask(model_name, image_filename, color_pallete='pascal_voc'):
  import time
  import mxnet as mx
  from mxnet import image
  from mxnet.gluon.data.vision import transforms
  from matplotlib import pyplot as plt
  import gluoncv
  from gluoncv.utils.viz import get_color_pallete
  import matplotlib.image as mpimg

  # using cpu
  ctx = mx.cpu(0) 

  # model download
  model = gluoncv.model_zoo.get_model(model_name, pretrained=True)

  tic = time.time()
  # image loading(원본이미지를 그대로 불러옴.)
  _, org_image = gluoncv.data.transforms.presets.rcnn.load_test(im_fname)
  img = mx.nd.array(org_image) 

  # image pre transforming
  from gluoncv.data.transforms.presets.segmentation import test_transform
  img = test_transform(img, ctx)

  # inference
  output = model.predict(img)
  predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

  # display
  mask = get_color_pallete(predict, color_pallete)
  mask.save('output.png')
  mask_img = image.imread('output.png')
  
  #mask image와 원본이미지 합치기 투명도 0.8
  mask_img = gluoncv.utils.viz.cv_merge_two_images(org_image, mask_img, alpha=0.8)

  # 좌측 원본 우측 합친 이미지 출력
  fig = plt.figure(figsize=(15,15))
  ax1 = fig.add_subplot(1,2,1)
  ax1 = gluoncv.utils.viz.plot_image(org_image, ax=ax1)
  ax1.set_title('original')
  ax1.axis("off")
  ax2 = fig.add_subplot(1,2,2)
  ax2 = gluoncv.utils.viz.plot_image(mask_img, ax=ax2)
  ax2.set_title('segmentation')
  ax2.axis("off")
  plt.show()
  
  print('process time : {}'.format(time.time() - tic))

### image test

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_semantic_segmentation_mask('fcn_resnet101_voc',im_fname, 'pascal_voc')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/karolmajek/Mask_RCNN/blob/master/images/8433365521_9252889f9a_z.jpg?raw=true')
display_semantic_segmentation_mask('psp_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation_mask('fcn_resnet101_voc',im_fname, 'pascal_voc')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation_mask('psp_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/detection/biking.jpg?raw=true')
display_semantic_segmentation_mask('deeplab_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://miro.medium.com/max/700/1*hO_md3xNtcncjAmGrTwxYA.jpeg')
display_semantic_segmentation_mask('fcn_resnet101_voc',im_fname, 'pascal_voc')

In [ ]:
im_fname = gluoncv.utils.download('https://miro.medium.com/max/700/1*hO_md3xNtcncjAmGrTwxYA.jpeg')
display_semantic_segmentation_mask('deeplab_resnet101_ade',im_fname, 'ade20k')

In [ ]:
im_fname = gluoncv.utils.download('https://miro.medium.com/max/700/1*hO_md3xNtcncjAmGrTwxYA.jpeg')
display_semantic_segmentation_mask('psp_resnet101_ade',im_fname, 'ade20k')